In [1]:
import torch
from torch import flatten
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import BatchNorm2d
from torchvision.transforms import ToTensor
import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
import optuna
from optuna.trial import TrialState

In [2]:
class CustomDataset(Dataset):
    def __init__(self):
        self.data = []

        target_954 = np.load('data_segment/Y_natur_954.npz')
        for j in range(3000):
            self.data.append(["data_segment/natur_X/954_" + str(j) + '.npz',target_954['arr_0'][j]])
#         print(self.data)

        target_9_7 = np.load('data_segment/Y_natur_9_7.npz')
#         print(target_9_7['arr_0'])
        for j in range(3000):
            self.data.append(["data_segment/natur_X/9_7_" + str(j) + '.npz',target_9_7['arr_0'][j]])

        
        target_Air = np.load('data_segment/Y_natur_Air.npz')
#         print(target_Air['arr_0'])
        for j in range(3000):
            self.data.append(["data_segment/natur_X/Air_" + str(j) + '.npz',target_Air['arr_0'][j]])
        
        temp = np.load('data_segment/natur_X/954_0.npz')
        self.img_dim = temp['arr_0'].shape  
#         print(self.img_dim)
        
        
    def __len__(self):
        return len(self.data)    
    
    
    def __getitem__(self, idx):
        img_path, target = self.data[idx]
        
        img = np.load(img_path)
        img_tensor = torch.from_numpy(img['arr_0'])
        
        target_tensor = torch.tensor(target)
#         print(target_tensor, target)
        
#         img_tensor = img_tensor.permute(2, 0, 1)
#         class_id = torch.tensor([class_id])
        
        return img_tensor, target_tensor

    
data = CustomDataset()
print(len(data))

print(data[1][0].size())
print(data[1][0].type())


9000
torch.Size([2, 64, 50])
torch.DoubleTensor


In [3]:
Train_split = 0.7
Val_split = 0.15
Test_split = 0.15
(trainData, valData, testData) = random_split(data, [Train_split, Val_split, Test_split],\
                                              generator=torch.Generator().manual_seed(42))
print(len(trainData))
print(len(valData))
print(len(testData))

6300
1350
1350


In [4]:
BATCH_SIZE = 32
trainDataLoader = DataLoader(trainData, shuffle=True, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)



# print(testDataLoader.dataset[0])

# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
print(trainSteps)

valSteps = len(valDataLoader.dataset) // BATCH_SIZE
print(valSteps)

testSteps = len(testDataLoader.dataset) // BATCH_SIZE
print(testSteps)


# print(trainDataLoader.dataset)
# print(trainDataLoader.dataset[1])
# print(trainDataLoader.dataset[1][0].type())
# print(trainDataLoader.dataset[1][1])

196
42
42


In [5]:
class LeNet(Module):
    def __init__(self):
        # call the parent constructor
        super(LeNet, self).__init__()

        self.norm1 = BatchNorm2d(2)

        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels = 2, out_channels = 8,\
                            kernel_size = (16, 20), stride = (16,10))#16 - height, 20 - width 
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) #stride = step
        
#         self.norm2 = BatchNorm2d(8)
        
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=32, out_features=8)
        self.relu2 = ReLU()
        self.fc2 = Linear(in_features=8, out_features=1)

    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
#         print(x)
        x = self.norm1(x)
    
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = flatten(x, 1)
        
        
        
        x = self.fc1(x)
        x = self.relu2(x)

        output = self.fc2(x)

        return output

In [6]:
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [22]:
# define training hyperparameters
INIT_LR = 0.001

# initialize the LeNet model
print("[INFO] initializing the LeNet model...")

model = LeNet().to(device)

if str(device) == 'cpu':
    model.type(torch.DoubleTensor)
elif str(device) == 'cuda':
    model.type(torch.cuda.DoubleTensor)
# model.type(torch.DoubleTensor)



print(model)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)

scheduler = ReduceLROnPlateau(opt, 'min')



lossMSE = nn.MSELoss()
# lossMAE = nn.L1Loss()

# initialize a dictionary to store training history
H = {"train_loss": [],"val_loss": []}


[INFO] initializing the LeNet model...
LeNet(
  (norm1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(2, 8, kernel_size=(16, 20), stride=(16, 10))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32, out_features=8, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)


In [23]:
EPOCHS = 10
print("[INFO] training the network...")


startTime = time.time()
# loop over our epochs
for e in range(0, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
   
    # loop over the training set
    for id_batch, (x, y) in enumerate(trainDataLoader):
#         print(id_batch)
#         print(y)
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
#         print(y.size(), y.type())
#         print(x.size(), x.type())
#         print(x)
        # perform a forward pass and calculate the training loss
        pred = model(x)
#         print(pred)
        y = torch.reshape(y,(len(y),1))

        loss = lossMSE(pred, y)
#         loss = lossMAE(pred, y)
        
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        totalTrainLoss += loss
    # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        for (x, y) in valDataLoader:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))
            # make the predictions and calculate the validation loss
            pred = model(x)
            y = torch.reshape(y,(len(y),1))
            totalValLoss += lossMSE(pred, y)
#             totalValLoss += lossMAE(pred, y)
    
    scheduler.step(totalValLoss)
    
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Val loss: {:.6f}".format(
        torch.sqrt(avgTrainLoss), torch.sqrt(avgValLoss)))
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: \
        {:.2f}s".format(endTime - startTime))

[INFO] training the network...
[INFO] EPOCH: 1/10
Train loss: 0.022098, Val loss: 0.005180
[INFO] EPOCH: 2/10
Train loss: 0.006979, Val loss: 0.006324
[INFO] EPOCH: 3/10
Train loss: 0.006999, Val loss: 0.004797
[INFO] EPOCH: 4/10
Train loss: 0.007321, Val loss: 0.006793
[INFO] EPOCH: 5/10
Train loss: 0.005448, Val loss: 0.002963
[INFO] EPOCH: 6/10
Train loss: 0.004452, Val loss: 0.004230
[INFO] EPOCH: 7/10
Train loss: 0.005244, Val loss: 0.002861
[INFO] EPOCH: 8/10
Train loss: 0.004630, Val loss: 0.003297
[INFO] EPOCH: 9/10
Train loss: 0.004569, Val loss: 0.004812
[INFO] EPOCH: 10/10
Train loss: 0.003596, Val loss: 0.002788
[INFO] total time taken to train the model:         117.91s


$$\bf OPTUNA$$

In [27]:
dict_activ = {'relu': nn.ReLU(), 'elu': nn.ELU(), 'logsigm': nn.LogSigmoid(), 'tanh': nn.Tanh()}

class ConvNet(Module):
    def __init__(self, trial):
        # call the parent constructor
        super(ConvNet, self).__init__()
        
        self.norm1 = BatchNorm2d(2)
        
        out_ch = trial.suggest_int("out_ch", 2, 16, log = True)
        self.conv1 = Conv2d(in_channels = 2, out_channels = out_ch,\
                            kernel_size = (16, 20), stride = (16,10))#16 - height, 20 - width 
        
        activ_1 = trial.suggest_categorical("activ_1", ['relu', 'elu', 'logsigm', 'tanh'])
        self.act1 = dict_activ[activ_1]
        
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) #stride = step
         
        # initialize first (and only) set of FC => RELU layers
        
        n_neur = trial.suggest_int("n_neur",2,128,log = True)
        self.fc1 = Linear(in_features = 4*out_ch, out_features = n_neur)
        
        
        activ_2 = trial.suggest_categorical("activ_2", ['relu', 'elu', 'logsigm', 'tanh'])
        self.act2 = dict_activ[activ_2]
        
        self.fc2 = Linear(in_features = n_neur, out_features = 1)

    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
#         print(x)
        x = self.norm1(x)
    
        x = self.conv1(x)
        x = self.act1(x)
        x = self.maxpool1(x)
        
        x = flatten(x, 1)
        
        
        
        x = self.fc1(x)
        x = self.act2(x)
        output = self.fc2(x)

        return output

In [28]:
EPOCHS = 10
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def objective(trial):

    # Generate the model.
    model = ConvNet(trial).to(device)
    
    if str(device) == 'cpu':
        model.type(torch.DoubleTensor)
    elif str(device) == 'cuda':
        model.type(torch.cuda.DoubleTensor)

    # Generate the optimizers.

    # try RMSprop and SGD
    '''
    optimizer_name = trial.suggest_categorical("optimizer", ["RMSprop", "SGD"])
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr,momentum=momentum)
    '''
    #try Adam, AdaDelta adn Adagrad
    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "Adadelta","Adagrad"])
    
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    
#     scheduler = ReduceLROnPlateau(optimizer, 'min')
    
    batch_size=trial.suggest_int("batch_size", 4, 256, log = True)
    
    criterion = nn.MSELoss()

    
    trainDataLoader = DataLoader(trainData, shuffle = True, batch_size = batch_size)
    valDataLoader = DataLoader(valData, batch_size = batch_size)
#     testDataLoader = DataLoader(testData, batch_size = batch_size)
    
    trainSteps = len(trainDataLoader.dataset) // batch_size
    valSteps = len(valDataLoader.dataset) // batch_size
    
    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
       
        totalTrainLoss = 0
        totalValLoss = 0
    
        for id_batch, (x, y) in enumerate(trainDataLoader):
            (x, y) = (x.to(device), y.to(device))
            
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, torch.reshape(y,(len(y),1)))
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss    

        model.eval()

        
        with torch.no_grad():
            for (x, y) in valDataLoader:
                # send the input to the device
                (x, y) = (x.to(device), y.to(device))
                # make the predictions and calculate the validation loss
                pred = model(x)
                totalValLoss += criterion(pred, torch.reshape(y,(len(y),1)))
    #             totalValLoss += lossMAE(pred, y)
        
        avgTrainLoss = totalTrainLoss / trainSteps
        avgValLoss = totalValLoss / valSteps
        
        
        trial.report(torch.sqrt(avgValLoss), epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avgValLoss

cpu


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

trial = study.best_trial

print('sqrt_MSE: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2022-12-06 15:43:03,325] A new study created in memory with name: no-name-91c64837-8d1d-4ab6-8d63-45b394f197e5
[I 2022-12-06 15:45:09,299] Trial 0 finished with value: 0.025910444083480463 and parameters: {'out_ch': 9, 'activ_1': 'elu', 'n_neur': 17, 'activ_2': 'relu', 'optimizer': 'Adagrad', 'lr': 9.32321115649712e-05, 'batch_size': 145}. Best is trial 0 with value: 0.025910444083480463.
[I 2022-12-06 15:46:49,729] Trial 1 finished with value: 0.0004588513120081346 and parameters: {'out_ch': 2, 'activ_1': 'elu', 'n_neur': 73, 'activ_2': 'elu', 'optimizer': 'Adadelta', 'lr': 0.0006989123919447076, 'batch_size': 11}. Best is trial 1 with value: 0.0004588513120081346.
[I 2022-12-06 15:48:30,097] Trial 2 finished with value: 0.00010066948366899458 and parameters: {'out_ch': 3, 'activ_1': 'elu', 'n_neur': 4, 'activ_2': 'elu', 'optimizer': 'Adam', 'lr': 2.796156953623737e-05, 'batch_size': 8}. Best is trial 2 with value: 0.00010066948366899458.
[I 2022-12-06 15:50:08,464] Trial 3 finishe